In [1]:
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

In [2]:
driver = GraphDatabase.driver("neo4j+s://100f53e5.databases.neo4j.io", auth=("neo4j", 'KJFr24ZSRZLfzloplVfwYh1Z0EttolS1BslIBo8-21g'))
driver

In [8]:
def create_and_return_friendship(tx):
    query = (
                "CREATE (p1:Person { name: $person1_name }) "
                "CREATE (p2:Person { name: $person2_name }) "
                "CREATE (p1)-[:KNOWS]->(p2) "
                "RETURN p1, p2"
            )
    result = tx.run(query, person1_name="Alice", person2_name="Ben")
    try:
        return [{"p1": row["p1"]["name"], "p2": row["p2"]["name"]}
                for row in result]
    except ServiceUnavailable as exception:
        print("{query} raised an error: \n {exception}".format(
            query=query, exception=exception))
        raise

In [9]:
with driver.session() as session:
    result = session.write_transaction(create_and_return_friendship)
    
    for row in result:
        print("Created friendship between: {p1}, {p2}".format(p1=row['p1'], p2=row['p2']))

Created friendship between: Alice, Ben


In [12]:
def find_and_return_person(tx):
    query = (
        '''
        Match (p:Person) where p.born > 1990 RETURN p limit 10
        '''
    )
    result = tx.run(query)
    return [row for row in result]

In [13]:
with driver.session() as session:
    result = session.read_transaction(find_and_return_person)
    for row in result:
        print("Found person: {row}".format(row=row))

Found person: <Record p=<Node id=44 labels=frozenset({'Person'}) properties={'born': 1996, 'name': 'Jonathan Lipnicki'}>>
